In [ ]:
#https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

###REQUIREMENTS

In [6]:
#LIBRARIES
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#UTILITY FUNCTIONS

In [7]:
#TIME RELATED
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [8]:
#VISULIZATIONS
import matplotlib.pyplot as plt
%matplotlib inline
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.savefig("loss_graph"+str(time.time())+".png")

FOR LOADING NATURAL LANGUAGE DATA FROM EXAMPLE

In [4]:
SOS_token = 0
EOS_token = 1

#LANGUAGE CLASS SHOULD BE THE SAME FOR 
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        #########################
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    
    #print(len(pairs))
    #print(pairs[0])
    #input()

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(lang1, lang2, reverse=False):
    #READLANG FUNC CALLED
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    #FILTEREDPAIRS Func CALLED
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        #ADD SENTENCE CALLED
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

#Nueral Network Functions
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['tu es fascinant .', 'you re fascinating .']


_________________
ADAPTED FOR CODE
_________________

In [9]:
#import code tokenizer
import sentencepiece as spm
class Tokenizer:

    def __init__(self, filepath='python_tokenizer.model'):
        self.sp = spm.SentencePieceProcessor(model_file=filepath)

    def encode(self, text, t=int):
        return self.sp.encode(text, out_type=t)

    def decode(self, pieces):
        return self.sp.decode(pieces)

    @staticmethod
    def train(input_file='data/raw_sents.txt', model_prefix='sp_model', vocab_size=30522):
        spm.SentencePieceTrainer.train(input=input_file, model_prefix=model_prefix, vocab_size=vocab_size,
                                       #input_sentence_size=2 ** 16, shuffle_input_sentence=True)
                                       input_sentence_size=number_of_lines, shuffle_input_sentence=True)
        
#instantiate tokenizer model
code_tokenizer = Tokenizer('../python_tokenizer.model')

In [11]:
#us
def tokenize_code(code):
    tokens = code_tokenizer.encode(code)
    code_tokens = []
    for t in tokens:
        literal = code_tokenizer.decode(t)
        if literal =='':
            literal = ' '
        code_tokens.append(literal)
    return code_tokens
 
#example
#code = data[1][0]
#print(tokenize_code(code))

In [12]:
#FOR LOADING CODE-COMMENT PAIRS DATA
SOS_token = 0
EOS_token = 1

#LANGUAGE CLASS SHOULD BE THE SAME FOR 
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence,code=False):
        if code:
            #for code we include white space... will that be a problem?
            for word in tokenize_code(sentence):
                self.addWord(word)
        else:
            for word in sentence.split(' '):
                self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
    # Read the file and split into lines

    pairs = read_code_commet_pairs('../code-comment-100-100.csv')
    
    input_lang = Lang('code')
    output_lang = Lang('comment')

    return input_lang, output_lang, pairs

MAX_LENGTH = 300

#DEPRICIATED FROM NL
'''
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]
'''

from csv import reader
def read_code_commet_pairs(file):
    with open(file, 'r') as read_obj:
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        # Iterate over each row in the csv using reader object
        pairs = []
        for row in csv_reader:
            # row variable is a list that represents a row in csv
            #row[0] - code
            #row[1] - comment #we have to normalize it (?)
            pairs.append([row[0],normalizeString(row[1])])
        read_obj.close()

    #print('num samples: ', len(pairs))
    return pairs
    
def prepareData(lang1, lang2, reverse=False,num_samples = 10000):
    #READLANG FUNC CALLED
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    
    #UNCOMMENT TO HAVE LIMITED SAMPLE SIZE
    pairs = pairs[1:num_samples+1]
    
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        #ADD SENTENCE CALLED
        input_lang.addSentence(pair[0],code=True)
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


#Nueral Network Functions
def indexesFromSentence(lang, sentence):
    try:
        if lang.name == 'code':
            return [lang.word2index[word] for word in tokenize_code(sentence)]
        #comment
        else:  
            return [lang.word2index[word] for word in sentence.split(' ')]
    except:
        print(lang.name)
        print(sentence)
        print(sentence.split(' '))
        input()

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


In [24]:
#are these differrent
#[input_lang.word2index[word] for word in tokenize_code(pairs[1][0])]
#[output_lang.word2index[word] for word in pairs[1][1].split(' ')]

In [13]:
#pairs = read_code_commet_pairs('../code-comment-300-300.csv')
num_samples = 1000
input_lang, output_lang, pairs = prepareData('code', 'comment', True, num_samples=num_samples)
#random.seed()
print(random.choice(pairs))

Reading lines...
Read 1000 sentence pairs
Counting words...
Counted words:
code 1589
comment 1594
["    def browser(self):\n        \n        return self.update('browser')\n", 'datastorebrowser used to browse this datastore .']


Nueral network LSTM (ie, our NMT)

In [14]:
#the Neural Network set up

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.randn(1, 1, self.hidden_size, device=device)
        #return torch.zeros(1, 1, self.hidden_size, device=device)
    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.randn(1, 1, self.hidden_size, device=device)
        #return torch.zeros(1, 1, self.hidden_size, device=device)
    
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        #return torch.zeros(1, 1, self.hidden_size, device=device)
        return torch.randn(1, 1, self.hidden_size, device=device)

In [22]:
#TRAINING MODEL UTILS

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    
    #dont have to go through each one
    #'''
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
    #'''
    #encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)
    
    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

#train model
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.1):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    
    #TODO: use different optimizer?
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    return plot_losses

In [20]:
#Configure our model and train it!
teacher_forcing_ratio = 0.5
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.001).to(device)



In [21]:
#TRAIN/FIT MODEL
#'''
plot_losses = trainIters(encoder1, attn_decoder1, 200, 
                         print_every=1, 
                         plot_every=1,
                         learning_rate=0.0001)
showPlot(plot_losses)
#'''

RuntimeError: input.size(-1) must be equal to input_size. Expected 256, got 4608

In [14]:
#try to save the model
#https://pytorch.org/tutorials/beginner/saving_loading_models.html
torch.save(encoder1.state_dict(), 'encoder1.model')
torch.save(attn_decoder1.state_dict(), 'attn_decoder1.model')

In [25]:
#LOAD THE MDOELS
encoder1.load_state_dict(torch.load('encoder1.model'))
attn_decoder1.load_state_dict(torch.load('attn_decoder1.model'))

RuntimeError: Error(s) in loading state_dict for EncoderRNN:
	size mismatch for embedding.weight: copying a param with shape torch.Size([4345, 256]) from checkpoint, the shape in current model is torch.Size([8072, 256]).

POST TRAINING EVALUATION

In [83]:
#BASIC MODEL USE
#output_words, attentions = evaluate(encoder1, attn_decoder1, "je suis trop froid .")
#plt.matshow(attentions.numpy())
#plt.savefig("mygraph.png")

In [17]:
#functions for evaluating translations
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

#gives us some examples of our model in action
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')
        
    

In [18]:
evaluateRandomly(encoder1, attn_decoder1,n=1)

>     def read1(self, n=None):
        
        self._unsupported("read1")


= read up to n bytes with at most one read system call .
< the the the . <EOS>



FOR VISULAIZING TRANSLATION WITH ATTENTION GRAPHS

In [19]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    
    #tokenize_code(sentence)
    #ax.set_xticklabels([''] + input_sentence.split(' ') +
    ax.set_xticklabels([''] + tokenize_code(input_sentence) +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    #plt.show()
    plt.savefig("attentionMap_"+str(time.time())+".png")


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)

evaluateAndShowAttention(random.choice(pairs)[0])

input =     def socket(self):
        
        return self.sock




output = the the the . <EOS>


In [ ]:
#USE MODEL ON TEST SET TO BE EVALUATED WITH BLEU AND OTHER NMT SCORES

In [15]:
import nltk
from nltk.translate.bleu_score import sentence_bleu 
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.chrf_score import sentence_chrf
from nltk.translate.gleu_score import sentence_gleu
from nltk.translate.meteor_score import single_meteor_score
from nltk.translate.nist_score import sentence_nist
from nltk.translate.ribes_score import sentence_ribes
from nltk.tokenize import word_tokenize
nltk.download('wordnet') #reqired for meteor
import pickle
import string
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon
from tqdm import tqdm

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/avielstein/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
#gives us some examples of our model in action
def evaluate_tansmetric_Randomly(encoder, decoder, n=10):
    for i in range(n):
        print('-----------')
        pair = random.choice(pairs)
        original = pair[1]
        print('original:',original)
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        #score = single_meteor_score(original, output_sentence)
        score = sentence_bleu(original, output_sentence, smoothing_function=SmoothingFunction().method1)
        print('generated:', output_sentence)
        print(score)
        #print('')
evaluate_tansmetric_Randomly(encoder1, attn_decoder1,n=10)

-----------
original: information about this alarm .
generated: the the the . <EOS>
0.014400124446705301
-----------
original: hardware configuration of the host . this might not be available for a disconnected host .
generated: the the the . <EOS>
0.015226277779914149
-----------
original: mirror test make tarball except filename is unicode .
generated: the the the . <EOS>
0.014400124446705301
-----------
original: a synonym for self .encode .
generated: the the the . <EOS>
0.01340082578177889
-----------
original: return string representation of a set . this looks like set list of elements .
generated: the the the . <EOS>
0.015226277779914149
-----------
original: set the current state of the encoder . state must have been returned by getstate .
generated: the the the . <EOS>
0.015226277779914149
-----------
original: return footnote link id .
generated: the the the . <EOS>
0.014400124446705301
-----------
original: a singleton managed object that provides methods for guest authentic